In [2]:
target_lang="en"  # change to your target lang
speaker = "M01"
model_name = "yip-i/torgo_xlsr_finetune-" + speaker
speaker_to_filter_out = ['MC01', 'MC02', 'MC03', 'MC04','FC01','FC02','FC03']


from datasets import load_dataset

username = "jindaznb"  # change to your username

dataset = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))

In [10]:
from huggingface_hub import notebook_login

!git config --global user.email "jindaz.work@outlook.com"
!git config --global user.name "j"

notebook_login()

In [6]:
from huggingface_hub import create_repo, Repository

try:
  repo = Repository(repo_path_local, clone_from=repo_path_remote)
  print(f"\nRepository {repo_path_remote} already exists.")
except:
  create_repo(repo_path_remote, exist_ok=True)
  repo = Repository(repo_path_local, clone_from=repo_path_remote)
  print("\nRepository created.")

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/europarl_bilingual_kenlm_3-gram is already a clone of https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram. Make sure you pull the latest changes with `repo.git_pull()`.
WARNING - /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/europarl_bilingual_kenlm_3-gram is already a clone of https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram. Make sure you pull the latest changes with `rep


Repository jindaznb/europarl_bilingual_kenlm_3-gram already exists.


In [2]:
europarl = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")
europarl

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2051014 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/zhang.jinda1/.cache/huggingface/datasets/jindaznb___parquet/jindaznb--en_corpora_parliament_processed-063cb73b5f63b162/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Dataset({
    features: ['text'],
    num_rows: 2051014
})

# train wordlevel ngram model

In [1]:
# !kenlm/build/bin/lmplz -o 3    --text europarl.txt     --arpa {repo_path_local}/output_model.klm_trigram.arpa     --discount_fallback --skip_symbols|     kenlm/build/bin/build_binary     -T /dev/stdin {repo_path_local}/output_model.klm_trigram.arpa

In [2]:
!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa" --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 51495092 types 127507
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1530084 2:10480902144 3:19651692544 4:31442706432 5:45853949952
Statistics:
1 127506 D1=0.648223 D2=1.00063 D3+=1.40113
2 3731487 D1=0.712145 D2=1.06726 D3+=1.38333
3 16334069 D1=0.801852 D2=1.11912 D3+=1.36491
4 31609785 D1=0.880996 D2=1.19342 D3+=1.38133
5 41878656 D1=0.91918 D2=1.23229 D3+=1.33838
Memory estimate for binary LM:
type      MB
probing 1904 assuming -p 1.5
probing 2200 assuming -r models -p 1.5
trie     892 without quantization
trie     488 assuming -q 8 -b 8 quantization 
trie     782 assuming -a 22 array pointer compression
trie     378 assuming -a 22 

In [1]:
!head -20 5gram.arpa

\data\
ngram 1=46
ngram 2=1430
ngram 3=27526
ngram 4=267526
ngram 5=1334843

\1-grams:
-3.0126686	<unk>	0
0	<s>	-4.5599556
-1.612649	</s>	0
-1.5884404	R	-1.4659708
-1.5655105	IH	-1.564043
-1.600376	Z	-1.5807759
-1.5544841	AH	-1.6780617
-1.5768242	M	-1.5517106
-1.5768242	P	-1.5669491
-1.5884404	SH	-1.442344
-1.5768242	N	-1.5247967
-1.5884404	V	-1.4767572


In [6]:
# with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
#   has_added_eos = False
#   for line in read_file:
#     if not has_added_eos and "ngram 1=" in line:
#       count=line.strip().split("=")[-1]
#       write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
#     elif not has_added_eos and "<s>" in line:
#       write_file.write(line)
#       write_file.write(line.replace("<s>", "</s>"))
#       has_added_eos = True
#     else:
#       write_file.write(line)

In [9]:
if ngram_order > 1:
  !kenlm/build/bin/build_binary {repo_path_local}/output_model.klm_trigram.arpa {repo_path_local}/output_model.klm_trigram.bin

Reading europarl_bilingual_kenlm_3-gram/output_model.klm_trigram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [41]:
repo.push_to_hub(commit_message="Upload n-gram model")

Upload file output_model.klm_trigram_raw.bin:   1%|          | 3.28k/519k [00:00<?, ?B/s]

To https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram
   0de756c..d62b286  main -> main

WARNING - To https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram
   0de756c..d62b286  main -> main



'https://huggingface.co/jindaznb/europarl_bilingual_kenlm_3-gram/commit/d62b2865e075145c34c988f7851aa6fa23f5ff3b'

In [1]:
from transformers import AutoProcessor
test_speaker = "M01"
model_user = "macarious"
model_repo = f"torgo_xlsr_finetune_{test_speaker}"
model_name = f"{model_user}/{model_repo}"
processor = AutoProcessor.from_pretrained(model_name)

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [4]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.
Only 46 unigrams passed as vocabulary. Is this small or artificial data?


In [5]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [7]:
from huggingface_hub import Repository

repo = Repository(local_dir="model_staging", clone_from=model_name)

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/macarious/torgo_xlsr_finetune_M01 into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.45k/1.18G [00:00<?, ?B/s]

Download file runs/Jan25_00-36-00_d1005/events.out.tfevents.1706161084.d1005.11790.0:  17%|#7        | 3.45k/1…

Download file runs/Jan25_00-36-00_d1005/1706161084.7326531/events.out.tfevents.1706161084.d1005.11790.1:  61%|…

Download file runs/Nov20_19-57-17_0f53531ac0df/events.out.tfevents.1700510385.0f53531ac0df.52655.0:  11%|#    …

...

Clean file runs/Nov20_19-05-29_0f53531ac0df/1700507598.2107837/events.out.tfevents.1700507598.0f53531ac0df.321…

Clean file runs/Nov20_19-57-17_0f53531ac0df/1700510385.488164/events.out.tfevents.1700510385.0f53531ac0df.5265…

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file runs/Nov20_19-57-17_0f53531ac0df/1700535469.8101287/events.out.tfevents.1700535469.0f53531ac0df.526…

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

In [8]:
processor_with_lm.save_pretrained("model_staging")

In [2]:
import pandas as pd

df = pd.read_csv('torgo.csv')
df['audio'] = df['audio'].apply(lambda x: f"/work/van-speech-nlp/data/torgo/{x}")
df.to_csv('torgo.csv', index=False)